### Análise dos dados

- Análise univariada dos dados
- Análise multivariada dos dados

In [ ]:
numericos = df_vendas_limpo.select_dtypes('number').columns.tolist()
categoricos = df_vendas_limpo.select_dtypes('object').columns.tolist()

print(f"Há {len(numericos)} dados numéricos, sendo eles:")
print(numericos)
print(f"\nHá {len(categoricos)} dados categóricos, sendo eles:")
print(categoricos)

Estatística descritiva

In [ ]:
df_vendas_limpo.describe().T.drop(labels=["InvoiceDate", "CustomerID"])

- A maioria dos valores dos produtos unicos são baixos tendo em 75% apenas 3.75.
- A quantidade de produtos comprados esta na mediana de 6 produtos por compra.
- 